In [41]:
#Importing Necessory Libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException , StaleElementReferenceException, ElementNotInteractableException 
from bs4 import BeautifulSoup
import time
import requests
from pathlib import Path
import pandas as pd

### 1.Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [6]:
#Define Function
def amazon_search(product):
    driver = webdriver.Chrome()
    try:
        driver.get('https://www.amazon.in/')
        time.sleep(2)
        #Xpath
        search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
        search_bar.send_keys(product)
        time.sleep(2)
        search_button = driver.find_element(By.XPATH,'//span[@id="nav-search-submit-text"]') #Xpath
        search_button.click()
        time.sleep(10)
    except NoSuchElementException:
        print('Exception Raised--')
if __name__ == "__main__":
    user_input = input("Enter Name Of product-- ")  #User input
    amazon_search(user_input)


Enter Name Of product-- fruits
Exception Raised--


### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [68]:
driver = webdriver.Chrome()
driver.get('https://www.amazon.in/')
time.sleep(2)
search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search_bar.send_keys('Guitar')
search_button = driver.find_element(By.XPATH,'//span[@id="nav-search-submit-text"]')
search_button.click()
time.sleep(2)
product_link = []
for page in range(0,3):
    print("Link Scaping From Pages - ",page+1)
    product_url = driver.find_elements(By.XPATH,'//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
    for i in product_url:
        product_link.append(i.get_attribute('href'))
        try:
            next_page = driver.find_element(By.XPATH,'//div[@class="a-section a-text-center s-pagination-container"]/span')
            
        except StaleElementReferenceException:
            print('Exception Raised:-')
            next_page1 = driver.find_element(By.XPATH,'//div[@class="a-section a-text-center s-pagination-container"]/span')
            next_page1.click()
Brand_Name = []
Name_of_the_Product = []
Price = []
Return_Exchange = []
Expected_Delivery = []
Availability = []

for i in product_link:
    driver.get(i)
    
    try:
        brand = driver.find_element(By.XPATH,'//tr[@class="a-spacing-small po-brand"]/td[@class="a-span9"]/span')
        Brand_Name.append(brand.text)
        
    except NoSuchElementException:
        Brand_Name.append('-')
    try:    
        name = driver.find_element(By.XPATH,'//div[@id="titleSection"]/h1/span')
        Name_of_the_Product.append(name.text)
        
    except NoSuchElementException:       
        Name_of_the_Product.append('-')
            
    try:    
        price = driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]//span[@class="a-price-whole"]')
        Price.append(price.text)
        
    except NoSuchElementException:
        Price.append('-')
       
    try:     
        exchange = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[5]/div[3]/div[4]/div[24]/div[2]/div/div/div/div[2]/div/ol/li[3]/div/span/div/span')
        Return_Exchange.append(exchange.text)
        
    except NoSuchElementException:       
        Return_Exchange.append('-')
       
    try:    
        delivery= driver.find_element(By.XPATH,'//div[@id="deliveryBlockMessage"]/div/div/span')
        Expected_Delivery.append(delivery.text)
        
    except NoSuchElementException:      
        Expected_Delivery.append('-')
         
    try:
        available= driver.find_element(By.XPATH,'//div[@id="availability"]/span')
        Availability.append(available.text)
        
    except NoSuchElementException:        
        Availability.append('-')

Link Scaping From Pages -  1
Link Scaping From Pages -  2
Link Scaping From Pages -  3


In [69]:
print(len(Brand_Name),len(Name_of_the_Product),len(Price),len(Return_Exchange),len(Expected_Delivery),len(Availability))
        

180 180 180 180 180 180


In [70]:
Amazon_Data = pd.DataFrame({"Brand_Name":Brand_Name,"Name_of_the_Product":Name_of_the_Product,"Price":Price,"Return_Exchange":Return_Exchange,"Expected_Delivery":Expected_Delivery,"Availability":Availability})
Amazon_Data

,Brand_Name,Name_of_the_Product,Price,Return_Exchange,Expected_Delivery,Availability
0,VAULT,Vault DA20 Dreadnought Acoustic Guitar With Gi...,"4,899",7 days Replacement,"FREE delivery Tuesday, 26 December. Order with...",In stock
1,VAULT,Vault Traveller 34 Inch Acoustic Guitar With T...,"4,299",7 days Replacement,"FREE delivery Tuesday, 26 December. Order with...",Only 1 left in stock.
2,Henrix,Henrix 38C 38 Inch Cutaway Acoustic Guitar Wit...,"3,099",7 days Replacement,"FREE delivery Tuesday, 26 December. Details",In stock
3,allmusicmart,"“ALL WOODEN” 38” INCH ""DOLPHIN"" Guitar Combo P...","3,550",7 days Replacement,"FREE delivery Monday, 25 December. Order withi...",In stock
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","1,999",7 days Replacement,"FREE delivery Monday, 25 December. Order withi...",In stock
...,...,...,...,...,...,...
175,Kadence,Kadence GenZ Guitar (with Guitar learning cour...,"5,999",7 days Replacement,"FREE delivery Sunday, 24 December. Details",In stock
176,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"8,999",7 days Replacement,"FREE delivery Friday, 29 December. Details",In stock
177,allmusicmart,"“ALL WOODEN” 38” INCH ""DOLPHIN""Guitar Combo Pa...","3,550",7 days Replacement,"FREE delivery Monday, 25 December. Order withi...",Only 1 left in stock.
178,allmusicmart,"“ALL WOODEN” 38” INCH ""DOLPHIN"" Guitar Combo P...","3,550",7 days Replacement,"FREE delivery Monday, 25 December. Order withi...",In stock


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import requests
from pathlib import Path
from IPython.display import display, Image

# Set up the Chrome webdriver
driver = webdriver.Chrome()

driver.get('https://www.google.com/')   
image_button = driver.find_element(By.XPATH, '//div[@class="gb_Ad gb_C gb_Wf gb_Nf"]/div[2]/a')
image_button.click()
time.sleep(2)

# Keywords
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for keyword in keywords:
    try:
        search_thing = driver.find_element(By.XPATH, '//textarea[@jsname="yZiJbe"]')
        search_thing.click()
        search_thing.send_keys(keyword)
    except NoSuchElementException:
        search_thing1 = driver.find_element(By.XPATH, '/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input')
        search_thing1.send_keys(keyword)

    search_button = driver.find_element(By.XPATH, '//button[@jsname="Tg7LZd"]')
    search_button.click()
    
    # Wait for some time to load search results
    time.sleep(2)
    
    # Find all images on the page using Selenium
    images = driver.find_elements(By.XPATH, '//img')
    
    # Create a directory for the keyword
    keyword_path = Path(keyword)
    keyword_path.mkdir(exist_ok=True)
    
    # Download the first 10 images for the current keyword
    for i, img in enumerate(images[:10], start=1):
        img_url = img.get_attribute('src')
        img_data = requests.get(img_url).content
        img_filename = f"{keyword}_{i}.jpg"
        img_path = keyword_path / img_filename
        with open(img_path, 'wb') as f:
            f.write(img_data)
        
        # Display the image in the Jupyter Notebook
        display(Image(filename=str(img_path)))
    
    # Clear the search bar for the next keyword
    search_word = driver.find_element(By.XPATH, '/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input')
    search_word.clear()

# Close the browser window
driver.quit()


### 4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.flipkart.com/')
time.sleep(2)
close_signin_tab = driver.find_element(By.XPATH,'//div[@class="JFPqaw"]/span').click()

search_bar = driver.find_element(By.XPATH,'//div[@class="_2SmNnR"]/input[@class="Pke_EE"]')
search_bar.send_keys('Oneplus Nord')

search_button = driver.find_element(By.XPATH,'//div[@class="_1sFryS _2alaMB"]/button').click()
product_link = []
product_url = driver.find_elements(By.XPATH,'//div[@class="_2kHMtA"]/a')
for i in product_url:
    product_link.append(i.get_attribute('href'))
time.sleep(2)
Brand_Name = []
Smartphone_Name = []
Colour = []
RAM = []
Storage_ROM = []
Primary_Camera = []
Secondary_Camera = []
Display_Size = []
Battery_Capacity = []
Price  = []
for i in product_link:
    driver.get(i)
    try:
        b_name = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')
        Brand_Name.append(b_name.text)
        
    except NoSuchElementException:
        Brand_Name.append('-')
    try:
        s_name = driver.find_element(By.XPATH,'//div[@class="_1AtVbE col-12-12"]/div/div/div[@class="_1UhVsV _3AsE0T"][1]/div/table/tbody/tr[@class="_1s_Smc row"][3]/td[@class="URwL2w col col-9-12"][1]')
        Smartphone_Name.append(s_name.text)
        
    except NoSuchElementException:
        Smartphone.append('-')
    try:
        clr = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(clr.text) 
        
    except NoSuchElementException:
        Colour.append('-')               
    try:
        rm = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(rm.text)
        
    except NoSuchElementException:
        RAM.append('-')
    try:
        strg = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage_ROM.append(strg.text)
        
    except NoSuchElementException:
        Storage_ROM.append('-')
        
    try:
        p_camera = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[1]/td[2]/ul/li')
        Primary_Camera.append(p_camera.text)
        
    except NoSuchElementException:
        Primary_Camera.append('-')
    try:
        s_camera =driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        Secondary_Camera.append(s_camera.text)
        
    except NoSuchElementException:
        Secondary_Camera.append('-')
    try:
        display = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_Size.append(display.text)
        
    except NoSuchElementException:
        Display_Size.append('-')
        
    try:
        battery = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[7]/table/tbody/tr/td[2]/ul/li') 
        Battery_Capacity.append(battery.text)
        
    except NoSuchElementException:
        Battery_Capacity.append('-')
    
    try:
        price = driver.find_element(By.XPATH,'//div[@class="CEmiEU"]/div[@class="_25b18c"]/div[1]')
        Price.append(price.text)
        
    except NoSuchElementException:
        Price.append('-')
        

In [73]:
print(len(Brand_Name),len(Smartphone_Name),len(Colour),len(RAM),len(Storage_ROM),len(Primary_Camera),len(Secondary_Camera),len(Display_Size),len(Battery_Capacity),len(Price))

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [ ]:
from selenium import webdriver
import time
import re

def get_coordinates(city):
    driver = webdriver.Chrome()  # Set up the Chrome webdriver
    driver.get("https://www.google.com/maps")  # Open Google Maps
    time.sleep(2)

    # Find the search box and enter the city
    search_box = driver.find_element_by_xpath("//input[@id='searchboxinput']")
    search_box.send_keys(city)
    search_box.submit()
    time.sleep(5)  # Wait for the search results to load

    # Get the URL which contains the coordinates
    current_url = driver.current_url

    # Extract latitude and longitude from the URL using regex
    coordinates = re.findall(r'@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)

    driver.quit()  # Close the browser window

    return coordinates[0] if coordinates else None

# Replace 'Your_City' with the city you want to search
city_name = 'New York'
coordinates = get_coordinates(city_name)

if coordinates:
    latitude, longitude = coordinates
    print(f"Geospatial coordinates for {city_name}: Latitude {latitude}, Longitude {longitude}")
else:
    print(f"Could not find coordinates for {city_name}")


## 6. Write a program to scrap all the available details of best gaming laptops from digit.in

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.digit.in/')
time.sleep(2)
search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div/div/div[2]/form/input[1]')
search_bar.send_keys('gaming laptop')

search_button = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div/div/div[2]/form/button').click()
laptop_url = []
laptoplink = driver.find_elements(By.XPATH,'//h3[@class=" text-clamp text-clamp-2"]/a')
for i in laptoplink:
    laptop_url.append(i.get_attribute('href'))    
#Create Empty List
laptop_name = []
operating_system = []
processor = []
storage = []
display_size = []
ram = []
laptop_weight = []
battery_backup = []
release_date = []
overall_rating = []
description = []
price = []
#Scrap Data Using Loop  For All Columns
for i in laptop_url:
    driver.get(i)
    try:
        lap_name = driver.find_element(By.XPATH,'//h1[@class="floatleft tabletblockdisplay pr20 "]')
        laptop_name.append(lap_name.text)
        
    except NoSuchElementException:
        laptop_name.append('-')
    
    try:
        os = driver.find_element(By.XPATH,'//div[@class="key_specifications "]/div/ul[1]/li[1]/div[1]/p[2]')
        operating_system.append(os.text)
        
    except NoSuchElementException:
        operating_system.append('-')
        
    try:
        pros = driver.find_element(By.XPATH,'//div[@class="key_specifications "]/div/ul[1]/li[2]/div[1]/p[2]')
        processor.append(pros.text)
        
    except NoSuchElementException:
        processor.append('-')
        
    try:
        strg = driver.find_element(By.XPATH,'//div[@class="key_specifications "]/div/ul[1]/li[3]/div[1]/p[2]')
        storage.append(strg.text)
        
    except NoSuchElementException:
        storage.append('-')
        
    try:
        display = driver.find_element(By.XPATH,'//div[@class="key_specifications "]/div/ul[1]/li[4]/div[1]/p[2]')
        display_size.append(display.text)
        
    except NoSuchElementException:
        display_size.append('-')
    try:
        rm = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div/div/div[2]/div[4]/div[3]/div[2]/table/tbody/tr[8]/td/table/tbody/tr[1]/td')
        ram.append(rm.text)
        
    except NoSuchElementException:
        ram.append('-')
    try:
        weight = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div/div/div[2]/div[4]/div[3]/div[2]/table/tbody/tr[10]/td/table/tbody/tr[2]/td')
        laptop_weight.append(weight.text)
        
    except NoSuchElementException:
        laptop_weight.append('-')
        
    try:
        battery = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div/div/div[2]/div[4]/div[3]/div[2]/table/tbody/tr[20]/td/table/tbody/tr/td')
        battery_backup.append(battery.text)
    
    except NoSuchElementException:
        battery_backup.append('-')
        
    try:
        date = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div/div/div[2]/div[1]/div[4]/div[1]/div[2]')
        release_date.append(date.text)
   
    except NoSuchElementException:
        release_date.append('-')
        
    try:
        rating = driver.find_element(By.XPATH,'//div[@class="rating_sec"]')
        overall_rating.append(rating.text)
        
    except NoSuchElementException:
        overall_rating.append('-')
    
    try:
        descp = driver.find_element(By.XPATH,'//div[@class="review-text"]/div/p')
        description.append(descp.text)
        
    except NoSuchElementException:
        description.append('-')
        
    try:
        prc = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div/div/div[2]/div[3]/div/div[3]/div[1]/p/span/bdi/span')
        price.append(prc.text)
    
    except NoSuchElementException:
        price.append('-')
            

### 7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [36]:
driver = webdriver.Chrome()
driver.get('http://www.forbes.com/')
driver.maximize_window()
time.sleep(10)
side_button = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div').click()
time.sleep(1)
slide_button = driver.find_element(By.XPATH,'//li[@class="TjJgrPSg cD45ib6e primary"]/div[@role="button"]').click()
time.sleep(2)
billionaires_list_tag = driver.find_element(By.XPATH,'//div[@class="_4fMm9"]/div/div[2]/div/a').click()
Top_Rank = []
Name =  []
Net_Worth = []
Age = []
Citizenship = []
Source = []
Industry = []

while True:
             
    rank = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[1]')
    for i in rank:
        Top_Rank.append(i.text)
        
    name = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[2]')    
    for i in name:
        Name.append(i.text)
        
    worth = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[3]')    
    for i in worth:
        Net_Worth.append(i.text)
    age = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[4]')    
    for i in age:
        Age.append(i.text)
    country = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[5]')
    for i in country:
        Citizenship.append(i.text)
    company = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[6]')
    for i in company:
        Source.append(i.text)
    indus  = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[7]')    
    for i in indus:
        Industry.append(i.text)       
    try:
        next_button1 = driver.find_element(By.XPATH,'//button[2][@class="Pagination_paginationBtn__UIBE-"]')
        next_button1.click()

    except:
        break
    

In [42]:
Billionaires_data = pd.DataFrame({"Top_Rank":Top_Rank,"Name":Name,"Net_Worth":Net_Worth,"Age":Age,"Citizenship":Citizenship,"Source":Source,"Industry":Industry})

2427 2427 2427 2427 2427 2427 2427


In [44]:
print(Billionaires_data)

     Top_Rank                      Name Net_Worth Age    Citizenship  \
0         202  Vincent Bolloré & family    $9.5 B  71         France   
1         202              Jim Pattison    $9.5 B  94         Canada   
2         204        Ernesto Bertarelli    $9.4 B  57    Switzerland   
3         204                 Wang Xing    $9.4 B  44          China   
4         206              Brian Chesky    $9.3 B  41  United States   
...       ...                       ...       ...  ..            ...   
2422     2540                   Yu Rong      $1 B  51          China   
2423     2540    Richard Yuengling, Jr.      $1 B  80  United States   
2424     2540             Zhang Gongyun      $1 B  60          China   
2425     2540    Zhang Guiping & family      $1 B  71          China   
2426     2540               Inigo Zobel      $1 B  66    Philippines   

                            Source               Industry  
0                      Investments  Finance & Investments  
1              

# 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [49]:
driver= webdriver.Chrome()
driver.get('https://www.youtube.com/')
driver.maximize_window()

In [50]:
search_bar = driver.find_element(By.XPATH,'//div[@id="search-input"]/input')
search_bar.send_keys('Maher Zain - Rahmatun Lil’Alameen')

In [51]:
search_button = driver.find_element(By.XPATH,'//button[@id="search-icon-legacy"]').click()

In [53]:
first_video = driver.find_element(By.XPATH,'//div[@id="title-wrapper"]/h3').click()
time.sleep(5)
for _ in range(200):
    driver.execute_script("window.scrollBy(2,000)")
# creating empty lists
comments = []
comment_time = []
Likes = []
# scrape comments
cm = driver.find_elements(By.XPATH,'//div[@id="body"]/div[2]/div[2]')
for i in cm[:500]:
    comments.append(i.text)
    
# scrape time when comment was posted
tm = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for i in tm[:500]:
    comment_time.append(i.text)
    
# scrape the comment likes
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer'][2]")
for i in like[:500]:
    Likes.append(i.text)
       

In [62]:
youtube_data = pd.DataFrame({"Comments":comments,"Comments Time":comment_time,"Comments Like":Likes})
youtube_data

,Comments,Comments Time,Comments Like
0,Proud that I'm Muslim. Thanks Allah Almighty.,2 weeks ago,127
1,Greetings from Romania. 🇷🇴 Praying for all Pal...,12 days ago (edited),43
2,I am proud to be a muslim from 🇧🇩\nAssalamuala...,1 month ago,340
3,I'm a Christian but I love to heard this song ...,2 weeks ago,278
4,حبيبي يا محمد يا رحمة للعالمين محمد ﷺ,2 weeks ago,92
...,...,...,...
495,Beautiful nasheed ..I am a Muslim ..thank you...,3 weeks ago,6
496,Masya allah SWT,1 month ago,2
497,Love you mohommad ﷺ,1 day ago,
498,Love you maher zain. You're amazing brother,13 days ago,1


# 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [75]:
# getting the web page of mentioned url
driver = webdriver.Chrome()
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(1)
driver.maximize_window()
time.sleep(2)
# locating the location search bar
search_bar = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input')
# entering London in search bar
search_bar.send_keys("London")
time.sleep(2)
# select London
London = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[2]/div/ul/li[2]").click()
# do click on Let's Go button
search_btn = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[5]/button[2]')
search_btn.click()
time.sleep(3)
#Scrap Data

Distance_City_Center =[]  #Create Empty List
distance_tag = driver.find_elements(By.XPATH,'//div[@class="property-distance"]')
for i in distance_tag:
    Distance_City_Center.append(i.text)
    
Dorm_From_Price = []          #Create Empty List
d_price_tag = driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[2]')
for i in d_price_tag:
    Dorm_From_Price.append(i.text)
    
Private_Price = []     #Create Empty List
p_price_tag = driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[1]')
for i in p_price_tag:
    Private_Price.append(i.text)
hostal_link = []     #Create Empty List
hostal_url = driver.find_elements(By.XPATH,'//div[@class="property-card"]/a') #XPATH of LINks
for i in hostal_url:
    hostal_link.append(i.get_attribute('href'))
Hostal_Name = []      #Create Empty List
Hostal_Description = []         #Create Empty List
Facilities = []                #Create Empty List
Over_All_Rating = []            #Create Empty List
Total_Review = []            #Create Empty List
for i in hostal_link:
    driver.get(i)
    
    try:
        property_name = driver.find_element(By.XPATH,'//div[@class="headline-container"]/div/h1')
        Hostal_Name.append(property_name.text)
    except NoSuchElementException:
        Hostal_Name.append('-')
    
    try:
        description_tag = driver.find_element(By.XPATH,'//div[@class="description-container"]')
        Hostal_Description.append(description_tag.text)
        
    except NoSuchElementException:
        Hostal_Description.append('-')
        
    try:
        f_tag = driver.find_element(By.XPATH,'//div[@class="facilities-container"]/ul/li/ul')
        Facilities.append(f_tag.text)
    except NoSuchElementException:
        Facilities.append('-')
    
    try:
        rating_tag = driver.find_element(By.XPATH,'//div[@class="rating rating-summary-container big clickable"][1]/div[1]')
        Over_All_Rating.append(rating_tag.text)
    except NoSuchElementException:
        Over_All_Rating.append('-')
    
    try:
        t_review = driver.find_element(By.XPATH,'//div[@class="rating rating-summary-container big clickable"][1]/div[2]/div[2]')
        Total_Review.append(t_review.text)
    except NoSuchElementException:
        Total_Review.append('-')
    

In [76]:
print(len(Distance_City_Center),len(Hostal_Name),len(Hostal_Description),len(Facilities),len(Dorm_From_Price),len(Over_All_Rating),len(Total_Review),len(Private_Price))

30 33 33 33 30 33 33 30
